In [5]:
import urllib
from bs4 import BeautifulSoup
from pytube import YouTube
import cv2
import os
import glob

## Manual Method

In [9]:
vid_urls = ['https://www.youtube.com/watch?v=N3pZ3S5ZZwk',
            'https://www.youtube.com/watch?v=xxXZ3aQwL_k',
            'https://www.youtube.com/watch?v=xqCgO24eDQ4',
            'https://www.youtube.com/watch?v=5J5u9_f4vE0',
            'https://www.youtube.com/watch?v=2JyRZW4nnLA',
            'https://www.youtube.com/watch?v=PDdeR-qolIM',
            'https://www.youtube.com/watch?v=xMcrzQ2D7o8',
            'https://www.youtube.com/watch?v=Fy4G9rDimvA',
            'https://www.youtube.com/watch?v=HpIM-ZwBVmw',
            'https://www.youtube.com/watch?v=Fq_cS8Xkjq0',
            'https://www.youtube.com/watch?v=9NZ_bbxkSBE',
            'https://www.youtube.com/watch?v=a99U82W2fYk',
            'https://www.youtube.com/watch?v=mJVPKIr1ygM',
            'https://www.youtube.com/watch?v=CIDLYHojOW0',
            'https://www.youtube.com/watch?v=MKEo_EbplOE',
            'https://www.youtube.com/watch?v=KKjWkP388eU',
            'https://www.youtube.com/watch?v=0LHreT0qFEI',
            'https://www.youtube.com/watch?v=IwUWRPpKYns',
            'https://www.youtube.com/watch?v=sw7NHLV2dsQ'

           ]

In [10]:

#where to save 
SAVE_PATH = 'youtube_scraping'
  
#link of the video to be downloaded 
for i in vid_urls: 
    try: 
          
        # object creation using YouTube
        # which was imported in the beginning 
        yt = YouTube(i) 
    except: 
          
        #to handle exception 
        print("Connection Error") 
      
    # get the video with the extension and
    # resolution passed in the get() function 

    try: 

        stream = yt.streams.get_by_itag(22)
        stream.download(SAVE_PATH)
        print('Task Completed!') 
    except: 
        try:
            stream = yt.streams.get_by_itag(135)
            stream.download(SAVE_PATH)
            print('Task Completed!') 
        except:
            try:
                stream = yt.streams.get_by_itag(18)
                stream.download(SAVE_PATH)
                print('Task Completed!')             
            except:
                print("Some Error!") 
print('Full Task Completed!') 

Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Task Completed!
Full Task Completed!


In [11]:
yt = YouTube('https://www.youtube.com/watch?v=FLM1LMuBs2o') 
yt.streams.filter(file_extension='mp4')

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="25fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="134" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="133" mime_type="video/mp4" res="240p" fps="25fps" vcodec="avc1.4d400c" progressive="False" type="video">, <Stream: itag="160" mime_type="video/mp4" res="144p" fps="25fps" vcodec="avc1.4d400b" progressive="False" type="video">, <Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">]

## Automated from Search Query- needs work

In [16]:
def get_urls(text, limit=10):
    '''Return list of youtube urls based on search query'''
    query = urllib.parse.quote(text)
    url = "https://www.youtube.com/results?search_query=" + query
    response = urllib.request.urlopen(url)
    html = response.read()
    soup = BeautifulSoup(html, 'html.parser')
    urls = []
    for i, vid in enumerate(soup.findAll(attrs={'class':'yt-uix-tile-link'})):
        if i < limit:
            urls.append('https://www.youtube.com' + vid['href'])
    print(f"Found {len(urls)} video links for {text}")
    return urls

def download_video(url, path=None, max_duration=10):
    '''download videos based on urls through pytube'''
      try:
        yt = YouTube(url)
        duration = int(yt.player_config_args['player_response']['streamingData']['formats'][0]['approxDurationMs'])
        if duration < max_duration*60*1000:
            yt = yt.streams.filter(file_extension='mp4').first()
            out_file = yt.download(path)
            file_name = out_file.split("\\")[-1]
            print(f"Downloaded {file_name} correctly!")
        else:
            print(f"Video {url} too long")
      except Exception as exc:
        print(f"Download of {url} did not work because of {exc}...")
        
def extract_images_from_video(video, folder=None, delay=30, name="file", max_images=20, silent=False):    
    '''use opencv to extract images from video file'''
    vidcap = cv2.VideoCapture(video)
    count = 0
    num_images = 0
    if not folder:
        folder = os.getcwd()
    label = max_label(name, folder)
    success = True
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    
    while success and num_images < max_images:
        success, image = vidcap.read()
        num_images += 1
        label += 1
        file_name = name + "_" + str(label) + ".jpg"
        path = os.path.join(folder, file_name)
        cv2.imwrite(path, image)
        if cv2.imread(path) is None:
            os.remove(path)
        else:
            if not silent:
                print(f'Image successfully written at {path}')
        count += delay*fps
        vidcap.set(1, count)
        
def extract_images_from_word(text, delete_video=False, image_delay=30, 
                             num_urls=10, max_images=20, name="youtube_scraping", max_duration=15, silent=False):
    '''main function to extract images from youtube videos based on search query'''
    
    if not os.path.exists(name):
        os.mkdir(name)
    urls = get_urls(text, num_urls)
    for url in urls:
        download_video(url, max_duration=max_duration)
    for i, video in enumerate(glob.glob("*.mp4")):
        extract_images_from_video(video, folder=name, delay=image_delay, name=name, max_images=max_images, silent=silent)
        if delete_video:
            os.remove(video)